In [5]:
#Imports

import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import StratifiedShuffleSplit

from __future__ import print_function

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
# This differs
from sklearn.neighbors import KNeighborsClassifier

print(__doc__)

Automatically created module for IPython interactive environment


In [6]:
# These paths are personal
test = pd.read_csv('/Users/imac/Documents/Tijdelijk/Machine Learning/EindProject/test.csv', delimiter=',')
train = pd.read_csv('/Users/imac/Documents/Tijdelijk/Machine Learning/EindProject/train.csv', delimiter=',')

In [7]:
def encode(train, test):
    le = LabelEncoder().fit(train.species) 
    labels = le.transform(train.species)           
    classes = list(le.classes_)                    
    test_ids = test.id                             
    
    train = train.drop(['species', 'id'], axis=1)  
    test = test.drop(['id'], axis=1)
    
    return train, labels, test, test_ids, classes

train, labels, test, test_ids, classes = encode(train, test)

In [8]:
sss = StratifiedShuffleSplit(labels, 10, test_size=0.2, random_state=23)

for train_index, test_index in sss:
    X_train, X_test = train.values[train_index], train.values[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

In [9]:
# Set the parameters to be tuned by cross-validation
tuned_parameters = [{'n_neighbors':[1, 10]}]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(KNeighborsClassifier(n_neighbors=3), tuned_parameters, cv=5,
                       scoring='%s_macro' % score)
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))

# Tuning hyper-parameters for precision

Best parameters set found on development set:

{'n_neighbors': 1}

Grid scores on development set:

0.886 (+/-0.043) for {'n_neighbors': 1}
0.721 (+/-0.048) for {'n_neighbors': 10}
# Tuning hyper-parameters for recall

Best parameters set found on development set:

{'n_neighbors': 1}

Grid scores on development set:

0.881 (+/-0.040) for {'n_neighbors': 1}
0.749 (+/-0.042) for {'n_neighbors': 10}
